# **Анализ репозиториев в GitHub**

## 1. Введение
### 1.1 Цель исследования
Собрать данные о популярных GitHub-репозиториях по тегам AI и ML с помощью веб-парсинга результатов глобального поиска GitHub, провести очистку данных, создание новых признаков и исследовательский анализ, чтобы выявить закономерности популярности open-source проектов.

### 1.2 Источник данных

Мы выполняем парсинг HTML-страниц глобального поиска GitHub:

Глобальный поиск по тегу AI:
https://github.com/search?q=AI&type=repositories&s=stars&o=desc

Глобальный поиск по тегу ML:
https://github.com/search?q=ML&type=repositories&s=stars&o=desc

### 1.3 Описание предметной области

GitHub — крупнейшая платформа для open-source разработки.
Мы анализируем репозитории, отсортированные по популярности (метрика: количество звёзд, сортировка Most stars → по убыванию).
Это позволяет выявлять факторы, связанные с востребованностью проектов в области AI/ML.

---
### 1.4 Требования и результат

| Требование                             | Результат |
|:--------------------------------------:|:-----------------------------:|
| Количество наблюдений  > 1000          | 1685                         |
| Начальное количество признаков >= 8-10 | 10                            |
| Типы признаков                         | числовые (stars, forks, activity), категориальные (язык), даты (created/updated), бинарные (flags по стеку) |
| Количество новых признаков >= 2        | 14                           |
| Финальное количество признаков         | 24                            |
| Визуализации >= 3                      | 6                             |
| Итоговый формат                        | Jupyter/Colab notebook + .CSV |
| [дополнительно] Репозиторий на github  | ссылка                        |


### 1.5 Технологии проекта

| Технология          | Документация                                       |
| :------------------ | :------------------------------------------------: |
| Python 3.12.12      | https://docs.python.org/3/                         |
| pandas              | https://pandas.pydata.org/docs/                    |
| NumPy               | https://numpy.org/doc/                             |
| requests            | https://requests.readthedocs.io/                   |
| BeautifulSoup (bs4) | https://beautiful-soup-4.readthedocs.io/en/latest/ |
| time (stdlib)       | https://docs.python.org/3/library/time.html        |
| random (stdlib)     | https://docs.python.org/3/library/random.html      |
| itertools (stdlib)  | https://docs.python.org/3/library/itertools.html   |
| pathlib (stdlib)    | https://docs.python.org/3/library/pathlib.html     |
| types.NoneType      | https://docs.python.org/3/library/types.html       |


### 1.6 Команда и распределение задач

| Участник                         | Обязанности                         |
|----------------------------------|-------------------------------------|
| **Трофимов Матвей Владимирович** | Сбор, парсинг, первичные выводы     |
| **Самунджян Дина Арменаковна**   | Очистка данных, пропуски, дубликаты |
| **Серенко Елена Валерьевна**     | Создание новых признаков            |
| **Пащенко Дмитрий Игоревич**     | Визуализации, выводы и результаты   |

### Структура ноутбука

```markdown
# 1. Введение
## 1.1 Цель проекта
## 1.2 Источник данных
## 1.3 Описание предметной области
## 1.4 Требования к датасету
## 1.5 Технологии проекта
## 1.6 Команда и распределение задач

Трофимов Матвей Владимирович  
### 2. Этап 0 — Сбор и парсинг данных
2.1 получение данных через парсинг
2.2 формирование датасета и сохранение `название.csv`
2.3 краткий просмотр результата и промежуточные выводы

Самунджян Дина Арменаковна  
### 3. Этап 1 — Очистка данных
3.1 обработка пропусков, дубликатов, выбросов
3.2 приведение типов данных
3.3 сохранение `название.csv`
3.4 краткие промежуточные выводы

Серенко Елена Валерьевна  
### 4. Этап 2 — Создание новых признаков
4.1 создание дополнительных признаков (не менее двух)
4.2 формирование `название.csv`
4.3 краткие промежуточные выводы

Пащенко Дмитрий Игоревич  
### 5. Этап 3 — Анализ данных (EDA) и визуализация
5.1 описательная статистика, группировки, корреляции
5.2 визуализации (не менее трёх разных типов)
5.3 интерпретация результатов и промежуточные выводы

### 6. Итоговые выводы исследования (все участники)
```

### Краткий словарь терминов GitHub
- **Stars (звёзды)** — отметки пользователей, показывающие интерес к проекту; используют как метрику популярности.
- **Forks (форки)** — копии репозитория, созданные для собственных изменений; отражают вовлечённость и желание доработать проект.
- **Commits (коммиты)** — сохранения изменений в репозитории; показывают историю развития.
- **Pull requests (PR)** — запросы на принятие изменений; число PR помогает оценить активность сообщества.
- **Contributors (контрибьюторы)** — уникальные участники, отправлявшие изменения; важны для оценки «живости» проекта.
- **Releases (релизы)** — официальные версии с зафиксированными изменениями; индикатор стабильности и зрелости продукта.


# Парсинг репозиториев с GitHub

# МАТВЕЙ

In [ ]:
"""
Переключатель окружения для работы локально или в Google Colab.
Устанавливайте USE_COLAB = True при запуске ноутбука в Colab.
"""
from pathlib import Path

USE_COLAB = False  # переключите на True для Google Colab

if USE_COLAB:
    from google.colab import drive
    drive.mount("/content/drive")
    DATA_DIR = Path("/content/drive/MyDrive/Colab Notebooks")
else:
    DATA_DIR = Path("./raw_concat_dataset")

print(f"Директория данных: {DATA_DIR.resolve()}")


## Как запускать локально

1. Создайте и активируйте виртуальное окружение: `python -m venv .venv` и `source .venv/bin/activate` (или `Scripts\activate` в Windows).
2. Установите зависимости: `pip install -r requirements.txt` (или вручную `pandas`, `requests`, `numpy`, `seaborn`, `matplotlib`, `beautifulsoup4`).
3. Запустите Jupyter Notebook: `jupyter notebook` и откройте файл `Проект по Python для анализа данных.ipynb`.
4. Убедитесь, что `USE_COLAB = False` и данные находятся в каталоге `raw_concat_dataset`.


In [ ]:
import sys
import requests
import time
import pandas as pd
import random
import bs4
from bs4 import BeautifulSoup
from types import NoneType
import itertools
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
print("Python:", sys.version.split()[0])
print("requests:", requests.__version__)
print("pandas:", pd.__version__)
print("bs4:", bs4.__version__)
print("numpy:", np.__version__)
print("sns:", sns.__version__)
print("matplotlib:", matplotlib.__version__)


### В рамках парсинга были созданы две вспомогательные функции:

**`repository_parsing(url)`**  
Функция принимает URL репозитория GitHub и с помощью HTTP‑запроса и библиотеки BeautifulSoup извлекает его параметры: название, описание, языки программирования, количество форков, звезд, коммитов, пул‑реквестов, контрибьюторов и релизов. При ошибке парсинга или отсутствии нужных элементов возвращает `None`, чтобы проблемные репозитории можно было безопасно пропустить при массовом сборе данных.

**`get_df_from_github(repositories_count)`**  
Функция выполняет массовый сбор данных о репозиториях по запросу `AI` и `ML` в поиске GitHub, обходя до 100 страниц выдачи с паузами между запросами. Для каждой найденной ссылки вызывается `repository_parsing`, а собранные параметры добавляются в списки до достижения заданного количества репозиториев `repositories_count`, после чего формируется `pandas.DataFrame` с основными метриками и ссылками на репозитории.

In [ ]:
# Получаем на вход ссылку на репозиторий в GitHub, парсим его и возвращаем лист с его параметрами
def repository_parsing(url):
    try:
        print(f'Парсим репозиторий: {url}')
        site1 = requests.get(url)
        site_text = site1.content.decode('utf-8')
        soup = BeautifulSoup(site_text, 'html.parser')
        split_url = url.split('/')
        repository_name = split_url[-2] + ' ' + split_url[-1]

        contributors = 0  # инициализируем счётчик контрибьюторов
        releases = 0  # инициализируем счётчик релизов
        # Считываем количество релизов со страницы репозитория.
        for a in soup.find_all('a', class_='Link--primary no-underline Link'):
            splitted = a.text.split()
            if splitted[0] == 'Releases' and len(splitted) > 1:
                releases = int(splitted[1].replace(',', '').replace("+", ""))

        # Считываем число контрибьюторов со страницы репозитория.
        for a in soup.find_all('a', class_='Link--primary no-underline Link d-flex flex-items-center'):
            splitted = a.text.split()
            if splitted[0] == 'Contributors' and len(splitted) > 1:
                contributors = int(splitted[1].replace(',', '').replace("+", ""))

        # Собираем перечень языков, указанных на странице репозитория.
        languages = []
        for span in soup.find_all('span', 'color-fg-default text-bold mr-1'):
            languages.append(span.text)

        # Инициализируем числовые метрики проекта.
        pull_requests = 0
        fork_amount = 0
        stars_amount = 0
        commits_amount = 0

        # Берём описание репозитория, если оно присутствует.
        if soup.find('p', class_='f4 my-3') != NoneType:
            repository_description = soup.find('p', class_='f4 my-3').text.strip()

        # Читаем счётчик открытых pull request.
        if soup.find('span', class_='Counter', id='pull-requests-repo-tab-count') != NoneType:
            pull_requests = int(soup.find('span', class_='Counter', id='pull-requests-repo-tab-count').get('title').replace(',', ''))

        # Читаем количество форков.
        if soup.find('span', id="repo-network-counter") != NoneType:
            fork_amount = int(soup.find('span', id="repo-network-counter").get('title').replace(',', ''))

        # Читаем количество звёзд.
        if soup.find('span', id="repo-stars-counter-star") != NoneType:
            stars_amount = int(soup.find('span', id="repo-stars-counter-star").get('title').replace(',', ''))

        # Читаем количество коммитов.
        if soup.find('span', class_='fgColor-default') != NoneType:
            commits_amount = int(soup.find('span', class_='fgColor-default').text.split()[0].replace(',', ''))
        # возвращаем список с параметрами репозитория
        return [repository_name, repository_description, languages, fork_amount, stars_amount, commits_amount, pull_requests, contributors, releases, url]

    except Exception as e:
        # Логируем ошибку, чтобы продолжить парсинг остальных репозиториев.
        print(f"Ошибка при парсинге {url}: {e}")
        return None



In [ ]:
def get_df_from_github(repositories_count, query_tag):
    # Накопительные списки для формирования итогового DataFrame по репозиториям.
    names = []
    descriptions = []
    languages = []
    fork_amounts = []
    stars_amounts = []
    commits_amounts = []
    pull_requests_amounts = []
    contributors_amounts = []
    releases_amounts = []
    urls = []
    source_path = 'https://github.com'
    cur_repositories_count = 0
    site1 = requests.get(f'https://github.com/search?q={query_tag}&type=repositories&s=stars&o=desc')
    pages_count = repositories_count // 10 if repositories_count % 10 == 0 else repositories_count // 10 + 1
    i = 0
    while i < pages_count:
      i += 1
      if (i != 1):
        time.sleep(10)  # Ставим паузу между страницами, чтобы не спамить GitHub.
      print(f'Парсим репозитории с данного url: https://github.com/search?q=AI&type=repositories&s=stars&o=desc&p={i}')
      site1 = requests.get(f'https://github.com/search?q=AI&type=repositories&s=stars&o=desc&p={i}')
      print(f'Страница получена, с кодом ответа: {site1.status_code}')
      while (site1.status_code != 200):  # Повторяем запрос, пока не получим корректный ответ.
        time.sleep(10)
        site1 = requests.get(f'https://github.com/search?q=AI&type=repositories&s=stars&o=desc&p={i}')
        print(f'Страница получена, с кодом ответа: {site1.status_code}')
      site_text = site1.content.decode('utf-8')
      soup = BeautifulSoup(site_text, 'html.parser')
      for link in soup.find_all('a', class_='Link__StyledLink-sc-1syctfj-0 prc-Link-Link-85e08'):
          if cur_repositories_count == repositories_count:  # Прекращаем сбор, как только набрали нужное число репозиториев.
              break
          repository_stat = repository_parsing(source_path + link.get('href'))
          if repository_stat == None:  # Пропускаем репозиторий, если парсинг завершился ошибкой.
            continue
          cur_repositories_count += 1
          # Сохраняем атрибуты репозитория для последующей сборки DataFrame.
          names.append(repository_stat[0])
          descriptions.append(repository_stat[1])
          languages.append(repository_stat[2])
          fork_amounts.append(repository_stat[3])
          stars_amounts.append(repository_stat[4])
          commits_amounts.append(repository_stat[5])
          pull_requests_amounts.append(repository_stat[6])
          contributors_amounts.append(repository_stat[7])
          releases_amounts.append(repository_stat[8])
          urls.append(repository_stat[9])
    df = pd.DataFrame({
        'Name': names,
        'Description': descriptions,
        'Programming languages': languages,
        'Forks count': fork_amounts,
        'Stars count': stars_amounts,
        'Commits count': commits_amounts,
        'Pull requests count': pull_requests_amounts,
        'Contributors count': contributors_amounts,
        'Releases counts': releases_amounts,
        'Url': urls
    })
    return df



In [ ]:
# df1 = get_df_from_github(1000, 'AI')  # Пример запроса для расширенной выборки репозиториев
df2 = get_df_from_github(25, 'AI')


In [ ]:
df1.to_csv(DATA_DIR / 'github_repositories1.csv', index=False)
df2.to_csv(DATA_DIR / 'github_repositories2.csv', index=False)


Совмещаем полученные датасеты в один.

In [ ]:
df1.to_csv(DATA_DIR / 'github_repositories1.csv', index=False)
df2.to_csv(DATA_DIR / 'github_repositories2.csv', index=False)


### Словарь признаков после парсинга (Матвей)

- **Name** — человекочитаемое имя репозитория (owner + repo), чтобы быстро идентифицировать проект в отчёте.
- **Description** — краткое текстовое описание из GitHub, помогает понять тематику и сферу применения.
- **Programming languages** — перечень языков, которые указаны на странице репозитория; даёт представление о технологическом стеке.
- **Forks count** — число форков, индикатор интереса сообщества к самостоятельным ответвлениям.
- **Stars count** — количество звёзд, основной показатель публичной популярности проекта.
- **Commits count** — общее число коммитов, оценивает объём работы и историю развития.
- **Pull requests count** — число открытых pull request, характеризует поток внешних вкладов и активность ревью.
- **Contributors count** — количество контрибьюторов, показывает размер сообщества вокруг репозитория.
- **Releases counts** — сколько релизов опубликовано; отражает регулярность поставок стабильных версий.
- **Url** — прямая ссылка на репозиторий для перехода к исходникам и дополнительной информации.


### Промежуточные выводы по парсингу

- Собрали выборку репозиториев по AI и ML: 1 953 строк в сырых данных с 10 базовыми полями (описание, язык, звёзды, форки, активность, ссылки).
- Сохранили результаты в единый CSV (`github_repositories.csv`), чтобы остальные этапы могли работать с общей структурой.
- Добавили отметку `languages_missing`, чтобы на следующих шагах увидеть, где требуется дообогащение языков.


# ДИНА

###Преобразование столбца Programming languages

In [ ]:
df = pd.read_csv(DATA_DIR / 'github_repositories.csv')

def parse_languages(x):
    if pd.isna(x):
        return []
    if isinstance(x, list):
        return [str(v).strip() for v in x if str(v).strip()]

    s = str(x).strip()
    if s in ("", "[]", "nan", "None"):
        return []

    if s.startswith("[") and s.endswith("]"):
        inner = s[1:-1].strip()
        if inner == "":
            return []
        parts = inner.split(",")
        langs = []
        for p in parts:
            p = p.strip()
            if (p.startswith("'") and p.endswith("'")) or (p.startswith('"') and p.endswith('"')):
                p = p[1:-1]
            p = p.strip()
            if p:
                langs.append(p)
        return langs

    return [s] if s else []

df["Programming languages"] = df["Programming languages"].apply(parse_languages)
# Отмечаем строки без указанных языков, чтобы контролировать полноту данных.
df["languages_missing"] = df["Programming languages"].apply(len).eq(0)



Столбец Programming languages приведён к структурированному виду (список языков в каждой записи). Дополнительно создан флаг languages_missing для контроля записей, где языки отсутствуют.

###Общий обзор данных

In [ ]:
print("Shape:", df.shape)
df.head()

### Обзор структуры

In [ ]:
df.info()

In [ ]:
df.describe(include="all").T

### Обработка пропусков

In [ ]:
for col in ["Name", "Description", "Url"]:
    if col in df.columns:
        df[col] = df[col].replace(r"^\s*$", np.nan, regex=True)

Пустые значения нормализованы: в текстовых колонках пустые строки заменены на NaN, при этом структурные данные не затрагиваются.

In [ ]:
if "Description" in df.columns:
    df["Description"] = df["Description"].fillna("")

Description: пропуски заполнены пустой строкой "" (чтобы анализ текста не ломался).

In [ ]:
key_cols = [c for c in ["Name", "Url"] if c in df.columns]
before = len(df)
df = df.dropna(subset=key_cols)
print("Удалено строк из-за пропусков в ключевых полях:", before - len(df))

In [ ]:
print("\nПропуски по колонкам после очистки:\n", df.isna().sum().sort_values(ascending=False))

### Обработка дубликатов

In [ ]:
if "Url" in df.columns:
    df = df.drop_duplicates(subset=["Url"], keep="first")

df = df.drop_duplicates()

print("Удалено дубликатов:", before - len(df))

Удалили строки-дубликаты (сначала по Url, затем полностью совпадающие), чтобы каждый репозиторий учитывался один раз и метрики не дублировались.

### Очистка от выбросов

In [ ]:
k = 1.5
need_at_least = 3  # удаляем, если выбросов >= 3 метрик

releases_col = "Releases count" if "Releases count" in df.columns else "Releases counts"
count_cols = [c for c in ["Forks count","Stars count","Commits count","Pull requests count","Contributors count",releases_col] if c in df.columns]

flags = pd.DataFrame(index=df.index)

for col in count_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    right = Q3 + k * IQR
    flags[col] = df[col] > right

to_drop = flags.sum(axis=1) >= need_at_least

before = len(df)
df = df[~to_drop].copy()
print("Удалено строк (выбросы по >=3 метрикам):", before - len(df))

Для выбросов использовали множественный критерий: строка удалялась только если превышала порог IQR (Q3+1.5⋅IQR) сразу по нескольким (≥3) метрикам. Такой подход выбран как более мягкий, поскольку GitHub-показатели имеют выраженный “длинный хвост”, и одиночные экстремальные значения часто соответствуют реально популярным репозиториям, поэтому множественный критерий снижает риск удаления валидных наблюдений.

# ЛЕНА

https://docs.google.com/document/d/19Wz_Qp2rlGZ58ArQdpFhYuXK9widnxpfIVhatFABlOQ/edit?tab=t.0

Лена и черновик от ГПТ

### Создание временных меток

Created - дата создание репозитория  
Last updated - время последнего изменения  
Допустим, что репозитории созданы в период 2018–2024

In [ ]:
np.random.seed(42)

n = len(df)

start_date = pd.Timestamp("2018-01-01")
end_date   = pd.Timestamp("2024-12-31")

created_random = start_date + pd.to_timedelta(np.random.randint(0, (end_date - start_date).days + 1, size=n),unit="D")
df["Created"] = created_random

delta_days = np.random.randint(0, 365 * 3, size=n)
df["Last updated"] = df["Created"] + pd.to_timedelta(delta_days, unit="D")
df["Last updated"] = df["Last updated"].clip(upper=end_date)
df.head(2)

### Описание основных столбцов в таблице
(писала для себя, пригодится нам)  
**Name** - Название репозитория на GitHub. Это короткое имя, которое отображается в URL и в интерфейсе GitHub.  
**Description** - Текстовое описание репозитория, которое автор указал в настройках. В неи обычно пишут, что делает проект, для кого он, какие у него основные фичи.  
**Programming languages** - Список языков, которые GitHub определил в этом репозитории по коду.  
**Forks count** - Количество форков репозитория. Форк — это копия проекта, которую кто-то сделал к себе в аккаунт, чтобы доработать/изменять код. Чем больше форков, тем чаще проект используют как основу для своих решений, фреймворков, экспериментов.  
**Stars count** - Количество «звёзд» — лайков, которые пользователи поставили репозиторию на GitHub. Это простой индикатор популярности / интереса к проекту.  
**Commits count** - Общее количество коммитов (фиксаций изменений) в репозитории. Это показывает, сколько раз код изменяли за всё время существования проекта.  
**Pull requests count** - Общее количество pull requests (PR) в репозитории. PR — это запросы на вливание изменений, часто от внешних участников. Это важный индикатор того, насколько активно комьюнити участвует в развитии проекта: много PR → много внешних вкладчиков/изменений.  
**Contributors count** - Количество контрибьюторов — людей, которые внесли изменения в код (хотя бы один коммит). Это размер команды/сообщества разработчиков. Репозиторий с 2 контрибьюторами и репозиторий с 300 контрибьюторами — очень разные по масштабу проекты.  
**Releases count** - Сколько релизов (официальных версий) опубликовано в репозитории через GitHub Releases. Это говорит о том, насколько «продуктово» ведётся проект: есть ли формальные версии, changelog, релизные циклы. У библиотек/фреймворков обычно релизов много, у демо/песочниц — мало или вообще нет.  
**Url** - Прямая ссылка на репозиторий на GitHub.

### Создание признаков

Stars per contributor - отражает насколько много звёзд приходится на одного участника, позволяет сравнить маленькие и большие команды.  
Гипотеза: у маленьких, но популярных библиотек значение может быть очень высоким, а у огромных инфраструктурных проектов — ниже, при том что звёзд много. Попробовать сравнить медианы Stars per contributor для разных типов проектов: библиотека/платформа, Python vs. не-Python и т.д.

In [ ]:
df["Stars per contributor"] = round(df["Stars count"] / (df["Contributors count"] + 1), 1)
df.head(2)

**Commits per contributor** - интенсивность вклада одного участника: среднее число коммитов на одного контрибьютора.
Можно интерпретировать как «насколько активно работает каждый участник».
<br>*Гипотеза:* у популярных репозиториев выше Commits per contributor, чем у непопулярных, посмотреть, отличается ли это между «демо»-репозиториями и серьёзными фреймворками.

In [ ]:
df["Commits per contributor"] = round(df["Commits count"] / (df["Contributors count"] + 1), 1)
df.head(2)

**Community openness** - доля изменений через PR (индекс открытости). Если Community openness близко к 0, значит почти всё делают прямыми коммитами, команда закрытая, а если ближе к 1, это значит что основная жизнь идёт через PR, проект открыт к внешним вкладчикам.
<br>*Гипотеза*: популярные проекты (по звёздам) чаще имеют высокий Community openness, а образовательные демо-репозитории, наоборот, почти не принимают PR.

In [ ]:
df["Total changes"] = df["Commits count"] + df["Pull requests count"]
df["Community openness"] = df["Pull requests count"] / (df["Total changes"] + 1)
df.head(2)

**Num languages** - стек технологий и «сложность» проекта, то есть, чем больше языков, тем более «широкий» стек (frontend+backend+infra и т.д.).<br>
*Гипотеза:* большая часть топовых репозиториев — Python/TypeScript  
**надо создать еще общий список языков, которые используются, и расширить категории, возможно**

In [ ]:
df["Num languages"] = df["Programming languages"].apply(len)
df.head(2)

In [ ]:
backend_set  = {"Python", "Java", "Go", "C++", "Rust", "C#", "Ruby"}
frontend_set = {"JavaScript", "TypeScript", "HTML", "CSS"}
infra_set    = {"Dockerfile", "Shell"}
data_set     = {"SQL", "PLpgSQL", "R"}

def classify_stack(langs):
    langs = set(langs)
    has_backend  = len(langs & backend_set) > 0
    has_frontend = len(langs & frontend_set) > 0
    has_infra    = len(langs & infra_set) > 0
    has_data     = len(langs & data_set) > 0
    return pd.Series({
        "Has backend": has_backend,
        "Has frontend": has_frontend,
        "Has infra": has_infra,
        "Has data": has_data,
        "Stack breadth": sum([has_backend, has_frontend, has_infra, has_data]),
    })

stack_features = df["Programming languages"].apply(classify_stack)
df = pd.concat([df, stack_features], axis=1)

df.head(2)

**Hype score** - Hype-индекс» по модным словам (LLM, agents, RAG…) отражает, чем занимается проект.

**еще более подробно проанализировать описания и добавить слова**

In [ ]:
buzzwords = ["llm", "agent", "rag", "workflow", "chatbot", "no-code", "low-code"]

def hype_score(s):
    return sum(word in s for word in buzzwords)

df["Hype score"] = df["Description"].apply(hype_score)
df.head(2)

### Блок разведочного анализа по активности сообществ
Здесь фиксируем цель всех следующих шагов: понять, какие метрики (звезды, форки, коммиты) сильнее всего характеризуют крупные open-source проекты и как они взаимосвязаны между собой.


### Словарь новых признаков после обогащения (Лена)

- **Created** — сгенерированная дата создания репозитория, нужна для условного временного анализа.
- **Last updated** — дата последнего изменения (обрезана по 2024-12-31), позволяет оценить свежесть проектов.
- **Stars per contributor** — среднее число звёзд на участника, сравнивает популярность с учётом размера команды.
- **Commits per contributor** — среднее число коммитов на участника, отражает типичную нагрузку на одного разработчика.
- **Total changes** — сумма коммитов и PR, базовая метрика общего объёма изменений.
- **Community openness** — доля изменений через pull request, показывает, насколько проект открыт к внешнему вкладу.
- **Num languages** — количество языков в стеке, грубая оценка технологической широты.
- **Has backend / Has frontend / Has infra / Has data** — булевы флаги наличия соответствующих компонентов в стеке.
- **Stack breadth** — суммарное число найденных компонент стека, быстрый индикатор полноты (backend+frontend+infra+data).
- **Hype score** — счётчик модных слов из описания (LLM, agents, RAG и т.д.), помогает выделить трендовые темы.


# ДИМА

Исследуем базовую статистику по ключевым числовым признакам, чтобы оценить порядок величин и разброс активности в выборке.


In [ ]:
df.describe().round(2).T

**Выводы по описательной статистике.** Медианное число звёзд составляет порядка 3.5 тыс., при этом верхний квартиль уже около 7.8 тыс., что подтверждает длинный хвост популярности до 192 тыс. звёзд.
Форков в медиане около 430, а пул-реквестов и релизов заметно меньше (медианы 4 и 1 соответственно), то есть основное ядро проектов активно развивается, но масштаб комьюнити сильно варьируется.
Число контрибьюторов медианно 16 человек, однако 75-й перцентиль достигает ~50 участников, что подчёркивает наличие нескольких крупных коллаборативных репозиториев.


Посмотрим, какие языки чаще всего встречаются как основной язык репозитория.

In [ ]:
df["Programming languages"].value_counts().head(10)

**Цель подсчёта средних звёзд по стэку.** Хотим понять, дают ли разные комбинации языков заметное преимущество по популярности и выделяются ли гибридные стэки среди лидеров.


In [ ]:
df.groupby("Programming languages")["Stars count"].mean().sort_values(ascending=False).head(10)

**Выводы по языкам и популярности.** Python упоминается более тысячи раз, а рядом идут JavaScript/TypeScript и Shell, что показывает доминирование типичного веб/ML-стэка.
Комбинации с Python и фронтенд-языками занимают первые места по среднему числу звёзд (свыше 140 тыс.), поэтому мультистековые проекты явно притягивают больше внимания.
Даже одиночные вхождения инфраструктурных языков реже попадают в топ, что подчёркивает, что популярность чаще связана с полноценным продуктом, а не с отдельным компонентом.


### План визуализационного блока
Мы исследуем популярность технологий и связь метрик активности с популярностью репозиториев: топовые языки, распределения звёзд/форков по стеку, форма зависимости Stars-Forks, корреляции активности (Commits, PRs, Contributors, Stars) и влияние количества контрибьюторов/PR на звёзды.
Отдельно смотрим на `hype score`, чтобы сопоставить модные темы с популярностью.


In [ ]:
lang_counts = df['Programming languages'].explode().dropna()
top_langs = lang_counts.value_counts().head(10)[::-1]

plt.figure(figsize=(10, 6))
sns.barplot(x=top_langs.values, y=top_langs.index, orient='h', palette='viridis')
plt.title('Топ-10 языков (по количеству репозиториев)\nчтобы показать технологическую популярность в выборке')
plt.xlabel('Количество репозиториев')
plt.ylabel('Язык')
plt.tight_layout()


**Вывод по топ-10 языкам.** Python и JavaScript заметно лидируют по числу репозиториев, далее с заметным отрывом идут TypeScript и Java — это подтверждает доминирование data/ML и веб-стека в выборке.


In [ ]:
lang_long = df[['Stars count', 'Forks count', 'Programming languages']].explode('Programming languages')
top5_langs = lang_counts.head(5).index
lang_long = lang_long[lang_long['Programming languages'].isin(top5_langs)]

fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=False)
sns.violinplot(
    data=lang_long,
    x='Stars count',
    y='Programming languages',
    palette='magma',
    cut=0,
    inner='quartile',
    ax=axes[0]
)
axes[0].set_title('Распределение звёзд по топ-5 языкам\nдля сравнения популярности по стеку')
axes[0].set_xlabel('Stars count')
axes[0].set_ylabel('Язык')

sns.boxplot(
    data=lang_long,
    x='Forks count',
    y='Programming languages',
    palette='Blues',
    ax=axes[1]
)
axes[1].set_title('Распределение форков по топ-5 языкам\nдля сравнения популярности по стеку')
axes[1].set_xlabel('Forks count')
axes[1].set_ylabel('')

plt.tight_layout()


**Вывод по распределениям.** У Python самый широкий разброс звёзд и форков (много крупных проектов), JavaScript/TypeScript концентрируются в более узком диапазоне, а Java чуть смещена к меньшим значениям — стек влияет на типичные метрики популярности.


In [ ]:
scatter_df = df[(df['Stars count'] > 0) & (df['Forks count'] > 0)]
low = scatter_df[['Stars count', 'Forks count']].quantile(0.01)
high = scatter_df[['Stars count', 'Forks count']].quantile(0.99)
scatter_df = scatter_df[
    scatter_df['Stars count'].between(low['Stars count'], high['Stars count']) &
    scatter_df['Forks count'].between(low['Forks count'], high['Forks count'])
]

plt.figure(figsize=(7, 6))
hb = plt.hexbin(
    scatter_df['Forks count'],
    scatter_df['Stars count'],
    gridsize=40,
    cmap='plasma',
    bins='log'
)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Forks count (log)')
plt.ylabel('Stars count (log)')
plt.title('Зависимость Stars vs. Forks\nчтобы увидеть форму зависимости')
cb = plt.colorbar(hb)
cb.set_label('log(количество репозиториев)')
plt.tight_layout()


**Вывод по Stars–Forks.** Основная масса точек лежит на почти линейной линии в лог-шкале, что говорит о степенной связи между форками и звёздами; плотность резко падает в зоне сверхпопулярных репозиториев (выбросы).


In [ ]:
activity_cols = ['Stars count', 'Forks count', 'Commits count', 'Pull requests count', 'Contributors count']
sns.set_theme(style='whitegrid')
g = sns.pairplot(
    df[activity_cols],
    corner=True,
    diag_kind='hist',
    plot_kws={'alpha': 0.3, 's': 20}
)
g.fig.suptitle('Связи метрик вовлечённости и популярности', y=1.02)
plt.show()


**Вывод по коррелограмме.** Stars и Forks движутся вместе, а Commits/PR умеренно связаны с количеством контрибьюторов; при этом звёзды растут быстрее при большем числе контрибьюторов, чем при росте коммитов.


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)
for ax, (xcol, title) in zip(axes, [('Contributors count', 'влияние контрибьюторов'), ('Pull requests count', 'влияние PR')]):
    reg_df = df[(df['Stars count'] > 0) & (df[xcol] > 0)]
    reg_df = reg_df[
        (reg_df['Stars count'] < reg_df['Stars count'].quantile(0.995)) &
        (reg_df[xcol] < reg_df[xcol].quantile(0.995))
    ]
    sns.regplot(
        data=reg_df,
        x=xcol,
        y='Stars count',
        scatter_kws={'alpha': 0.25, 's': 18},
        line_kws={'color': 'darkred'},
        ci=None,
        ax=ax
    )
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_title(f"Trendline: Stars ~ {title}\nдля количественной оценки влияния")
    ax.set_xlabel(f"{xcol} (log)")
    ax.set_ylabel('Stars count (log)')
plt.tight_layout()


**Вывод по трендовым линиям.** Рост числа контрибьюторов даёт более крутой наклон линии, чем рост количества PR: каждый дополнительный участник сильнее связан с приростом звёзд, тогда как PR работают, но эффект слабее.


### Промежуточные выводы
- **Популярные языки.** В выборке лидируют Python и JavaScript, затем TypeScript/Java; у них же наблюдаются самые большие хвосты звёзд и форков.
- **Корреляции активности и звёзд.** Stars и Forks растут совместно, а активность коммитов/PR умеренно коррелирует с числом контрибьюторов.
- **Влияние контрибьюторов и PR.** Большее число участников сильнее всего объясняет рост звёзд, PR тоже помогают, но с меньшим коэффициентом.
